# Metrics
- measuring different phases
- panda: format, save, plotting
- clustering

In [1]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt

In [2]:
import itk
import itkwidgets
from itkwidgets import view, compare

In [ ]:
# read superpixel image
img = 

In [ ]:
import math
from skimage.measure import regionprops, label
labeled = label(image2)
regions = regionprops(labeled)

## Measuring different phases

In [3]:
here


myViewer = view(concrete, mode='z') #slicing over z
myViewer

NameError: name 'here' is not defined

In [ ]:
class LaMisura:

    def __init__(self,inputs,outputs): 
        '''Input and output paths '''
        self.inputp = inputs #path
        self.outputp = outputs #path
def measurements(self,img):
        '''Extract area and circularity from binary input'''
        labeled = label(img)
        regions = regionprops(labeled)
        pixelSize = 1 #check this with Tonya/Rana
        larea=[]
        lcircularity=[]
        labels = np.arange(np.max(labeled) + 1)
        for region in regions:
            area = float(region.area * pixelSize**2)
            circularity = 4*math.pi*(region.area/region.perimeter**2)
            if not (100 <= area <= 10000
                and circularity > 0.5):
                labels[region.label] = 0
            else:
                larea.append(area)
                lcircularity.append(circularity)
        finalMask = labels[labeled] >0       
        sampleMetr = pd.DataFrame({
                            'area':larea,
                            'circularity':lcircularity
                            })
        return sampleMetr
        
    def histMeasurements(self,sampleMetr,mybin=10):
        '''Show histogram of area and circularity'''
        fig = plt.figure(figsize = (15,2.5))
        ax = fig.gca()
        sampleMetr.hist(bins=mybin,density=False,facecolor='g', alpha=0.75,ax = ax)

In [ ]:
#These are the coordinates of your crop
myViewer.roi_region()
#[789, 1102, 0], [700, 681, 31]

In [ ]:
myViewer.roi_slice()

In [ ]:
# User-selection is input for slicing np array
np_concrete = itk.array_from_image(concrete) 
roi = myViewer.roi_slice()
np_coreSample = np_concrete[roi]
#view(np_coreSample, annotations=False, ui_collapsed=True)

In [ ]:
np_coreSample.shape

In [ ]:
coreSample = itk.GetImageFromArray(np.ascontiguousarray(np_coreSample))
coreSample = itk.median_image_filter(coreSample)
view(coreSample)

In [ ]:
itk.imwrite(coreSample,'roiFiltered.tiff')

## Voxel clustering
- recreating volume with the least possible intensity values

In [ ]:
from skimage import img_as_ubyte
np_coreSample = itk.array_from_image(coreSample)
np_coreSample = np_coreSample[:,::2,::2] #downsample for quick testing
np_coreSample = (np_coreSample - np.min(np_coreSample)) / np.max(np_coreSample-np.min(np_coreSample))
np_coreSample = img_as_ubyte(np_coreSample)

In [ ]:
#plt.imshow((np_coreSample[1,:,:]))
view(np_coreSample)

In [ ]:
x, y, z = np_coreSample.shape
np_coreSampFlat = np_coreSample.reshape((-1,1))
np_coreSampFlat.shape

In [ ]:
len(np.unique(np_coreSampFlat))

In [ ]:
from sklearn import cluster
import time
start_time = time.time()

numberOfClusters = 3
kmeans_cluster = cluster.KMeans(n_clusters=numberOfClusters)
kmeans_cluster.fit(np_coreSampFlat)
cluster_centers = kmeans_cluster.cluster_centers_
cluster_labels = kmeans_cluster.labels_

duration = time.time() - start_time
print(f"Duration {duration} seconds")

In [ ]:
img_labels = cluster_centers[cluster_labels].reshape(x, y, z)

In [ ]:
plt.imshow(img_labels[15,:,:]) #cmp with np_coreSample

In [ ]:
#TODO: using img_as_ubyte here is a hack... 
img_labels = img_as_ubyte( (img_labels-np.min(img_labels))/np.max(img_labels-np.min(img_labels)) )
view(itk.GetImageFromArray(img_labels))

In [ ]:
view(img_labels,annotations=False, ui_collapsed=True,mode='z')

## Segment phases
- matrix, aggregate and interstitial space (pores)

In [ ]:
# TODO: show individual phases, apply morphological filters for cleaning